In [ ]:
!pip install video_diffusion_pytorch
!pip install einops

In [ ]:
!pip install kaggle
%cd "/content/"
!mkdir CCBDAHW1
%cd /content/CCBDAHW1


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [69]:
gpu_info = !nvidia-smi 
gpu_info = '\n'.join(gpu_info) 
if gpu_info.find('failed') >= 0: 
    print('Not connected to a GPU') 
else: 
    print(gpu_info)

Tue Jan 10 11:31:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    32W /  70W |  12594MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd /content/CCBDAHW1/
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/CCBDAHW1/"
!kaggle competitions download -c ccbda-2022-hw1
!unzip /content/CCBDAHW1/ccbda-2022-hw1.zip

In [5]:
# Import需要的套件
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time
from natsort import natsorted 
from google.colab.patches import cv2_imshow
from tqdm.auto import tqdm
import random
from sklearn.model_selection import train_test_split
from torch.utils import data
import copy
from torch.cuda.amp import autocast, GradScaler
from einops import rearrange
from pathlib import Path
from torch.optim import Adam
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tqdm import tqdm
from functools import partial
import torch.nn.functional as F
from torch.utils import data
import os
from torchvision import transforms as T, utils
import torch


In [6]:
#5 16 32 38
def make_dataset(data_path):
    samples = []
    for i in ['5', '16', '32', '38']:
        for j in os.listdir(data_path + "/" + i):
            video_path = data_path + "/" + i + "/" + j
            samples.append(video_path)
    return samples

IMG_SIZE = 32
Frame_size = 8
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


In [ ]:
[range(0,int(10),(int(10)//3))]

[range(0, 10, 3)]

In [7]:
class ActiontrainDataset(Dataset):
    def __init__(self,path,tfm):
      super(Dataset, self).__init__()
      self.path = path
      self.samples = make_dataset(self.path)
      self.transform = tfm
        
    def __len__(self):
      return len(self.samples)

    def load_video_tor(self,vid_path,use_transform):
      cap = cv2.VideoCapture(vid_path)
      frames = []
      try:
          while True:
              ret, frame = cap.read()
              if not ret:
                pad_num = Frame_size-len(frames)
                if pad_num > 0:
                  for j in range(pad_num):
                    frames.append(frames[int(len(frames)-1)])
                if pad_num <=0:
                  frames = frames[int((len(frames)//2)-Frame_size/2):int((len(frames)//2)+Frame_size/2)]
                break
              frame = crop_center_square(frame)
              frame = cv2.resize(frame, (IMG_SIZE,IMG_SIZE))
              #frame = frame[:, :, [2, 1, 0]]
              if use_transform is not None:
                frame = use_transform(frame)
              frames.append(frame)
          #frames_out = frames_out[::2]
      finally:
          cap.release()
      return rearrange(torch.stack(frames,dim = 0), 'f c h w -> c f h w')

    def __getitem__(self,idx):
      video_path = self.samples[idx]
      video = self.load_video_tor(video_path,self.transform)
      
      return video

In [8]:
from torch.utils.data.sampler import SubsetRandomSampler
from sys import path
from torchvision.transforms import Compose, ToTensor
from torch.utils.data import DataLoader
batch_size = 32

train_tfm = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset =ActiontrainDataset("/content/CCBDAHW1/train", tfm = train_tfm)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0, shuffle=True)


In [ ]:
train_dataset[539].shape

Train

In [9]:
def num_to_groups(num, divisor):
    groups = num // divisor
    remainder = num % divisor
    arr = [divisor] * groups
    if remainder > 0:
        arr.append(remainder)
    return arr
class EMA():
    def __init__(self, beta):
        super().__init__()
        self.beta = beta

    def update_model_average(self, ma_model, current_model):
        for current_params, ma_params in zip(current_model.parameters(), ma_model.parameters()):
            old_weight, up_weight = ma_params.data, current_params.data
            ma_params.data = self.update_average(old_weight, up_weight)

    def update_average(self, old, new):
        if old is None:
            return new
        return old * self.beta + (1 - self.beta) * new
CHANNELS_TO_MODE = {
    1 : 'L',
    3 : 'RGB',
    4 : 'RGBA'
}

def seek_all_images(img, channels = 3):
    assert channels in CHANNELS_TO_MODE, f'channels {channels} invalid'
    mode = CHANNELS_TO_MODE[channels]

    i = 0
    while True:
        try:
            img.seek(i)
            yield img.convert(mode)
        except EOFError:
            break
        i += 1
def gif_to_tensor(path, channels = 3, transform = T.ToTensor()):
    img = Image.open(path)
    tensors = tuple(map(transform, seek_all_images(img, channels = channels)))
    return torch.stack(tensors, dim = 1)

def identity(t, *args, **kwargs):
    return t

def cast_num_frames(t, *, frames):
    f = t.shape[1]

    if f == frames:
        return t

    if f > frames:
        return t[:, :frames]

    return F.pad(t, (0, 0, 0, 0, 0, frames - f))
def video_tensor_to_gif(tensor, path, duration = 120, loop = 0, optimize = True):
    images = map(T.ToPILImage(), tensor.unbind(dim = 1))
    first_img, *rest_imgs = images
    first_img.save(path, save_all = True, append_images = rest_imgs, duration = duration, loop = loop, optimize = optimize)
    return images

In [10]:
import torch
from video_diffusion_pytorch import Unet3D, GaussianDiffusion, Trainer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Unet3D(
    dim = 64,
    use_bert_text_cond = False,  # this must be set to True to auto-use the bert model dimensions
    dim_mults = (1, 2, 4, 8),
).to(device)


diffusion = GaussianDiffusion(
    model,
    image_size = 32,    # height and width of frames
    num_frames = 8,     # number of video frames
    timesteps = 1000,  # number of steps
    text_use_bert_cls = False,  
    loss_type = 'l1'    # L1 or L2
).to(device)

In [11]:
from torch.optim import Adam
batch = 32
ema = EMA(0.995)
ema_model = copy.deepcopy(diffusion)
train_data = train_data_loader
optimizer = Adam(diffusion.parameters(), lr = 1e-4)
scaler = GradScaler(enabled = True)

In [12]:
data = torch.load('/content/drive/MyDrive/Final Project/data/MSRVTT/videos/results/model_notext-current.pt')

step = data['step'] # epoochs 改(step +1,500)
diffusion.load_state_dict(data['model'])
ema_model.load_state_dict(data['ema'])
scaler.load_state_dict(data['scaler'])

In [ ]:
epochs = 500
for epoch in range(step +1,1000):
  
  for train_idx in tqdm(train_data):
    videos = train_idx
    with autocast(enabled = True):
      loss = diffusion(
          videos.to(device),
          cond = None
      )
      scaler.scale(loss).backward()
  print(f"Epoch: {epoch}, Loss: {loss.item():.3f}")
  scaler.step(optimizer)
  scaler.update()
  optimizer.zero_grad()

  if epoch % 10 == 0:
    if epoch < 2000:
      ema_model.load_state_dict(diffusion.state_dict())
    ema.update_model_average(ema_model, diffusion)

  if epoch != 0 and epoch % 5 == 0:
    milestone = epoch // 5
    num_samples = 4
    batches = num_to_groups(num_samples, batch)

    all_videos_list = list(map(lambda n: ema_model.sample(batch_size=n), batches))
    all_videos_list = torch.cat(all_videos_list, dim = 0)

    all_videos_list = F.pad(all_videos_list, (2, 2, 2, 2))

    one_gif = rearrange(all_videos_list, '(i j) c f h w -> c f (i h) (j w)', i = 2)
    video_path = f'/content/drive/MyDrive/Final Project/data/MSRVTT/videos/results_surf/{milestone}.gif'
    video_tensor_to_gif(one_gif, video_path)

    data = {
    'step': epoch,
    'model': diffusion.state_dict(),
    'ema': ema_model.state_dict(),
    'scaler': scaler.state_dict()
    }
    torch.save(data, '/content/drive/MyDrive/Final Project/data/MSRVTT/videos/results/model_notext-current.pt')
    
    print(f'Succefully save model-{milestone}')

In [68]:
num_samples = 1
batches = num_to_groups(num_samples, batch)

for i in range(100):
  all_videos_list = list(map(lambda n: ema_model.sample(batch_size=n), batches))
  all_videos_list = torch.cat(all_videos_list, dim = 0)
  one_gif = rearrange(all_videos_list, '(i j) c f h w -> c f (i h) (j w)', i = 1)
  video_path = f'/content/drive/MyDrive/Final Project/data/MSRVTT/videos/results_surf/output_gif/gif{i}.gif'
  video_tensor_to_gif(one_gif, video_path)

sampling loop time step:  46%|████▌     | 457/1000 [00:21<00:25, 21.45it/s]


KeyboardInterrupt: ignored

for FID

In [17]:
from torchvision.utils import save_image, make_grid

In [ ]:
num_samples = 1
batches = num_to_groups(num_samples, batch)

for i in range(100):
  all_videos_list = list(map(lambda n: ema_model.sample(batch_size=n), batches))
  all_videos_list = torch.cat(all_videos_list, dim = 0)

  #all_videos_list = F.pad(all_videos_list, (2, 2, 2, 2))

  one_gif = rearrange(all_videos_list[0,], 'c f h w -> f c h w')
  os.mkdir(f'/content/drive/MyDrive/Final Project/data/MSRVTT/videos/results_surf/output/video{i}/')
  for j in range(8):
    video_path = f'/content/drive/MyDrive/Final Project/data/MSRVTT/videos/results_surf/output/video{i}/frame{j}.jpg'
    save_image(one_gif[j,:] ,video_path)

In [36]:
for i in range(750+790+793+759):
  ori_vid = rearrange(train_dataset[i], 'c f h w -> f c h w')
  os.mkdir(f'/content/CCBDAHW1/videos/video{i}/')
  for j in range(8):
    video_path = f'/content/CCBDAHW1/videos/video{i}/frame{j}.jpg'
    save_image(ori_vid[j,:] ,video_path)

In [22]:
one_gif[1,:].shape

torch.Size([3, 32, 32])

In [31]:
!find /content/CCBDAHW1/train/5/ -name '*.mp4' | wc -l
!find /content/CCBDAHW1/train/16/ -name '*.mp4' | wc -l
!find /content/CCBDAHW1/train/32/ -name '*.mp4' | wc -l
!find /content/CCBDAHW1/train/38/ -name '*.mp4' | wc -l

750
790
793
759


In [53]:
import shutil

In [57]:
for i in range(100):
  shutil.copyfile(f"/content/drive/MyDrive/Final Project/data/MSRVTT/videos/results_surf/output/video{i}/frame2.jpg", f"/content/drive/MyDrive/Final Project/data/MSRVTT/videos/results_surf/output2/img{i}-2.jpg")


In [58]:
for i in range(3092):
  shutil.copyfile(f"/content/CCBDAHW1/videos/video{i}/frame2.jpg", f"/content/CCBDAHW1/image/img{i}-2.jpg")


In [63]:
!python /content/calc_metrics_for_dataset.py \
 --real_data_path "/content/CCBDAHW1/videos" \
 --fake_data_path "/content/drive/MyDrive/Final Project/data/MSRVTT/videos/results_surf/output" \
 --mirror 1 --gpus 1 --resolution 32 --metrics fvd2048_8f --verbose 10 --use_cache 0

In [61]:
import zipfile
from zipfile import ZipFile
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')  # 針對 Colab 改變路徑，本機環境可不用

with zipfile.ZipFile('test.zip', mode='w') as zf:
    for i in range(3092):
      zf.write(f"/content/CCBDAHW1/image/img{i}-2.jpg")

In [62]:
!pip install pytorch_gan_metrics
!python -m pytorch_gan_metrics.calc_metrics \
--path "/content/drive/MyDrive/Final Project/data/MSRVTT/videos/results_surf/output2" \
--stats "/content/drive/MyDrive/Colab Notebooks/test.zip"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.8/dist-packages/pytorch_gan_metrics/calc_metrics.py", line 26, in <module>
    (IS, IS_std), FID = get_inception_score_and_fid(
  File "/usr/local/lib/pyth